# Setup

In [1]:
import os
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
from random import randint
import math
from datetime import datetime
import pickle

from sklearn.linear_model import LogisticRegression
from sklearn import metrics
from sklearn.metrics import roc_curve
from sklearn.model_selection import KFold
from sklearn.model_selection import train_test_split
from sklearn.metrics import precision_recall_fscore_support
from sklearn.metrics import classification_report
from sklearn.metrics import precision_recall_curve
from sklearn.metrics import roc_curve
from sklearn.metrics import balanced_accuracy_score
from sklearn.metrics import confusion_matrix
from sklearn.metrics import accuracy_score
from sklearn.metrics import roc_auc_score

In [2]:
os.getcwd()

'D:\\Github\\Stomach-Status-Classification\\Experiment\\Approach\\LogisticRegression'

In [3]:
for i in range(3):
    os.chdir("..")

In [4]:
os.getcwd()

'D:\\Github\\Stomach-Status-Classification'

In [5]:
main_data_dir = os.getcwd() + "\\Data set"
kmean_data_dir = main_data_dir + "\\kmean_dataset"
kmean_data_10cv_dir = kmean_data_dir + "\\10cv_64"
import glob

kmean_data_10cv_lst = [x for x in glob.glob(kmean_data_10cv_dir + '\\*') if 'akaze' in x and 'index' in x]
print(kmean_data_10cv_lst)

['D:\\Github\\Stomach-Status-Classification\\Data set\\kmean_dataset\\10cv_64\\akaze_test_0_3_0.001_4_4histo_64_cv_index0.csv', 'D:\\Github\\Stomach-Status-Classification\\Data set\\kmean_dataset\\10cv_64\\akaze_test_0_3_0.001_4_4histo_64_cv_index1.csv', 'D:\\Github\\Stomach-Status-Classification\\Data set\\kmean_dataset\\10cv_64\\akaze_test_0_3_0.001_4_4histo_64_cv_index2.csv', 'D:\\Github\\Stomach-Status-Classification\\Data set\\kmean_dataset\\10cv_64\\akaze_test_0_3_0.001_4_4histo_64_cv_index3.csv', 'D:\\Github\\Stomach-Status-Classification\\Data set\\kmean_dataset\\10cv_64\\akaze_test_0_3_0.001_4_4histo_64_cv_index4.csv', 'D:\\Github\\Stomach-Status-Classification\\Data set\\kmean_dataset\\10cv_64\\akaze_test_0_3_0.001_4_4histo_64_cv_index5.csv', 'D:\\Github\\Stomach-Status-Classification\\Data set\\kmean_dataset\\10cv_64\\akaze_test_0_3_0.001_4_4histo_64_cv_index6.csv', 'D:\\Github\\Stomach-Status-Classification\\Data set\\kmean_dataset\\10cv_64\\akaze_test_0_3_0.001_4_4histo_64

In [6]:
kmean_train_paths = [x for x in kmean_data_10cv_lst if 'train' in x]
kmean_test_paths = [x for x in kmean_data_10cv_lst if 'test' in x]
print(kmean_train_paths)
print()
print(kmean_test_paths)
print(len(kmean_train_paths))
print(len(kmean_test_paths))

['D:\\Github\\Stomach-Status-Classification\\Data set\\kmean_dataset\\10cv_64\\akaze_train_0_3_0.001_4_4histo_64_cv_index0.csv', 'D:\\Github\\Stomach-Status-Classification\\Data set\\kmean_dataset\\10cv_64\\akaze_train_0_3_0.001_4_4histo_64_cv_index1.csv', 'D:\\Github\\Stomach-Status-Classification\\Data set\\kmean_dataset\\10cv_64\\akaze_train_0_3_0.001_4_4histo_64_cv_index2.csv', 'D:\\Github\\Stomach-Status-Classification\\Data set\\kmean_dataset\\10cv_64\\akaze_train_0_3_0.001_4_4histo_64_cv_index3.csv', 'D:\\Github\\Stomach-Status-Classification\\Data set\\kmean_dataset\\10cv_64\\akaze_train_0_3_0.001_4_4histo_64_cv_index4.csv', 'D:\\Github\\Stomach-Status-Classification\\Data set\\kmean_dataset\\10cv_64\\akaze_train_0_3_0.001_4_4histo_64_cv_index5.csv', 'D:\\Github\\Stomach-Status-Classification\\Data set\\kmean_dataset\\10cv_64\\akaze_train_0_3_0.001_4_4histo_64_cv_index6.csv', 'D:\\Github\\Stomach-Status-Classification\\Data set\\kmean_dataset\\10cv_64\\akaze_train_0_3_0.001_4_4

In [7]:
def to_categorical(y, num_classes=None, dtype='float32'):
    """to_categorical _summary_

    Arguments:
        y -- The label of the data set with the shape of [None, 1]

    Keyword Arguments:
        num_classes -- The num_classes in the data set (default: {None})
        dtype -- the type of each element of the label after reshape (default: {'float32'})

    Returns:
        the label of the data set with the shape of [number of samples, number of classes].
    """    
    y = np.array(y, dtype='int')
    input_shape = y.shape
    if input_shape and input_shape[-1] == 1 and len(input_shape) > 1:
        input_shape = tuple(input_shape[:-1])
    y = y.ravel()
    if not num_classes:
        num_classes = np.max(y) + 1
    n = y.shape[0]
    categorical = np.zeros((n, num_classes), dtype=dtype)
    categorical[np.arange(n), y] = 1
    output_shape = input_shape + (num_classes,)
    categorical = np.reshape(categorical, output_shape)
    return categorical

In [8]:
class roc_curve_score:
    def __init__(self, y_true, y_pred, num_class):
        self.y_test = to_categorical(y_true)
        self.y_score = to_categorical(y_pred)
        print(np.unique(self.y_test), np.unique(self.y_score))
        self.fprs = {}
        self.tprs = {}
        self.thresh_holds = {}
        for x in range(num_class):
            self.fprs[x], self.tprs[x], self.thresh_holds[x] = roc_curve(self.y_test[:, x], self.y_score[:, x], drop_intermediate=False)
            self.fprs[x] = self.fprs[x].tolist()
            self.tprs[x] = self.tprs[x].tolist()
            self.thresh_holds[x] = self.thresh_holds[x].tolist()
        
        self.fpr_micro_avg, self.tpr_micro_avg, self.threshold_micro_avg, = roc_curve(self.y_test.ravel(), self.y_score.ravel())
        
        all_fpr = np.unique(np.concatenate([self.fprs[i] for i in range(num_class)]))
        mean_tpr = np.zeros_like(all_fpr)
        for i in range(num_class):
            mean_tpr += np.interp(all_fpr, self.fprs[i], self.tprs[i])
        mean_tpr /= num_class
        self.fpr_macro_avg = all_fpr.tolist()
        self.tpr_macro_avg = mean_tpr.tolist()
    
    def get_tpr(self, _class = None):
        if(_class):
            return self.tprs[_class]
        else:
            return self.tprs
    
    def get_fpr(self, _class = None):
        if(_class):
            return self.fprs[_class]
        else:
            return self.fprs

    def get_thresholds(self, _class = None):
        if(_class):
            return self.thresh_holds[_class]
        else:
            return self.thresh_holds

    def get_roc_dict(self):
        return {
            "tpr" : self.get_tpr(),
            "fpr" : self.get_fpr(),
            "thresholds" : self.get_thresholds(),
            "fpr_micro_avg" : self.fpr_micro_avg.tolist(),
            "tpr_micro_avg" : self.tpr_micro_avg.tolist(),
            "fpr_macro_avg" : self.fpr_macro_avg,
            "tpr_macro_avg" : self.tpr_macro_avg,
        }

In [9]:
class utils:
    def __init__(self, confusion_matrix, y_true, y_pred):
        """
            - confusion_matrix: 2x2 numpy array
            - y_true: array of label
            - y_pred: array of output value calculated by model
            - fold_count: number of folds
        """
        
        # Initilize all indicator
        self.TP = confusion_matrix[0][0] # true positive
        self.FN = confusion_matrix[1][0] # false negative
        self.FP = confusion_matrix[1][0] # false positive
        self.TN = confusion_matrix[1][1] # true negative
        self.precision = self.TP/(self.FN +  self.TP) # Precision Score - Positive Predictive Value
        self.recall = self.TP / (self.TP + self.FN)
        self.f1_score =  (2 * self.TP) / (2 * self.TP + self.FP + self.FN)
        self.sensitivity = self.TP / (self.TP + self.FN) # True Positive Rate
        self.specificity = self.TN / (self.TN + self.FP) # False Positive Rate
        self.negative_predictive_value = self.TN / (self.TN + self.FN) # Negative Predictive Value
        self.false_negative_rate = self.FN / (self.FN + self.TP) # False Negative Rate
        self.false_positive_rate = self.FP / (self.FP + self.TN) # False Positive Rate
        self.false_discovery_rate = self.FP / (self.FP + self.TP) # False Discovery Rate
        self.false_omission_rate = self.FN / (self.FN + self.TN) # False Ommision Rate
        self.positive_likelihood_ratio = self.sensitivity / self.false_positive_rate # Positive Likelihood Ratio
        self.negative_likelihood_ratio = self.false_negative_rate / self.specificity # Negative Likelihood Ratio
        self.prevalence_threshold = math.sqrt(self.false_positive_rate) / (math.sqrt(self.sensitivity) + 
                                                                      math.sqrt(self.false_positive_rate)) # Prevalance Threshold
        self.threat_score = self.TP / (self.TN + self.FN + self.FP) # Threat Score
        self.prevalence = (self.TP + self.FN)/(self.TP + self.FN + self.TN + self.FP) # Prevalance 
        #  Matthews correlation coefficient
        self.matthews_correlation_coefficient = (self.TP*self.TN - self.FN*self.FP) / ((self.TP + self.FP)
                                                                                       *(self.TP + self.FN)
                                                                                       *(self.TN + self.FP)
                                                                                       *(self.TN + self.FN))
        self.fowlkes_mallows_index = math.sqrt(self.sensitivity + self.precision) # Fowlkes–Mallows index
        self.informedness = self.sensitivity + self.specificity - 1 # informedness
        self.markedness = self.precision + self.negative_predictive_value - 1 # markedness
        self.diagnostic_odds_ratio = self.positive_likelihood_ratio / self.negative_likelihood_ratio # Diagnostic odds ratio
        self.accuracy = (self.TP + self.TN) / (self.TP + self.TN + self.FP + self.FN)
        self.balanced_accuracy = (self.sensitivity + self.specificity) / 2
        self.roc_auc_macro = roc_auc_score(y_true, y_pred)
        self.roc_auc_micro = roc_auc_score(y_true, y_pred, average = 'micro')
        self.roc_auc_weighted = roc_auc_score(y_true, y_pred, average = 'weighted')
        self.cls_report = classification_report(y_true, y_pred)
        
        # Initilize the structure of output_dicts
        self.confusion_matrix = {
            "TP" : self.TP,
            "TN" : self.TN,
            "FN" : self.FN,
            "FP" : self.FP,
            "precision" : self.precision,
            "recall" : self.recall,
            "f1_score" : self.f1_score,
            "sensitivity" : self.sensitivity,
            "specificity" : self.specificity,
            "negative_predictive_value" : self.negative_predictive_value,
            "false_negative_rate" : self.false_negative_rate,
            "false_positive_rate" : self.false_positive_rate,
            "false_discovery_rate" : self.false_discovery_rate,
            "false_omission_rate" : self.false_omission_rate,
            "Positive_likelihood_ratio" : self.positive_likelihood_ratio,
            "Negative_likelihood_ratio" : self.negative_likelihood_ratio,
            "prevalence_threshold" : self.prevalence_threshold,
            "threat_score" : self.threat_score,
            "Prevalence" : self.prevalence,
            "Matthews_correlation_coefficient" : self.matthews_correlation_coefficient,
            "Fowlkes_Mallows_index" : self.fowlkes_mallows_index,
            "informedness" : self.informedness,
            "markedness" : self.markedness,
            "Diagnostic_odds_ratio" : self.diagnostic_odds_ratio,
            "accuracy" : self.accuracy,
            "balanced_accuracy" : self.balanced_accuracy
        }
        
        self.roc_auc_score = {
            "Macro": self.roc_auc_macro, 
            "Micro": self.roc_auc_micro,
            "Weight": self.roc_auc_weighted
        }
        
#         self.roc_curve = roc_curve_score(y_true=y_true, y_pred=y_pred, num_class=2).get_roc_dict()
        
        self.sub_dict = {
            "Confusion Matrix" : self.confusion_matrix,
            "ROC_AUC_SCORE" : self.roc_auc_score,
            "Classification Report" : classification_report(y_true.tolist(), y_pred.tolist(), 
                                                            labels = [0, 1], # 0 : Licit, 1 : Illicit
                                                            output_dict = True),
#             "ROC_DRAW" : self.roc_curve
        }
    def get_value(self):
        return self.sub_dict
        
# Test 
y_true = np.array([randint(0,1) for x in range(200)])
y_pred = np.array([randint(0,1) for x in range(200)])
confusion_matrix_test = confusion_matrix(y_true, y_pred)
base_utils = utils(confusion_matrix_test, y_true, y_pred)
base_utils.get_value()

{'Confusion Matrix': {'TP': 48,
  'TN': 41,
  'FN': 59,
  'FP': 59,
  'precision': 0.4485981308411215,
  'recall': 0.4485981308411215,
  'f1_score': 0.4485981308411215,
  'sensitivity': 0.4485981308411215,
  'specificity': 0.41,
  'negative_predictive_value': 0.41,
  'false_negative_rate': 0.5514018691588785,
  'false_positive_rate': 0.59,
  'false_discovery_rate': 0.5514018691588785,
  'false_omission_rate': 0.59,
  'Positive_likelihood_ratio': 0.7603358149849517,
  'Negative_likelihood_ratio': 1.3448826077045817,
  'prevalence_threshold': 0.5341959185602692,
  'threat_score': 0.3018867924528302,
  'Prevalence': 0.5169082125603864,
  'Matthews_correlation_coefficient': -1.3215127958773692e-05,
  'Fowlkes_Mallows_index': 0.9472044455566301,
  'informedness': -0.14140186915887853,
  'markedness': -0.14140186915887853,
  'Diagnostic_odds_ratio': 0.5653547831083022,
  'accuracy': 0.42995169082125606,
  'balanced_accuracy': 0.42929906542056073},
 'ROC_AUC_SCORE': {'Macro': 0.44499999999999

In [10]:
class Training:
    def __init__(self, fold_count, X, y):
        self.fold_count = fold_count
        self.param_grid = {
            "penalty" : ["l2"],
            "fit_intercept" : [True, False],
            "class_weight" : ["balanced", None],
            "solver" : ["newton-cg", "lbfgs", "liblinear", "sag", "saga"]
        }
        self.kf = KFold(n_splits=fold_count)
        self.history = {}
        self.X = X
        self.y = y
    
    def get_fold_value(self):
        return self.kf        
        
    def training(self, penalty, 
                 fit_intercept, 
                 class_weight, 
                 solver, train_case):
    
        # output_dict initilize
        output_dict = {}
    
        # poiter track the index of fold
        fold_index = 0
    
        for train_index, test_index in zip(self.X, self.y):
            train_df = pd.read_csv(train_index)
            test_df = pd.read_csv(test_index)
            
            print("\tFold: {}".format(fold_index))
            print("\tTRAIN:", train_index, "\n\tTEST:", test_index)
        
            # folding data
            x_train, y_train = train_df.iloc[:, 2:-1], train_df.iloc[:, -1]
            x_test, y_test = test_df.iloc[:, 2:-1], test_df.iloc[:, -1]
    
            # Training
            print("\t\tTraining : {}".format(fold_index), end = " -- ")
            print("Start: {}".format(datetime.now().strftime("%m/%d/%Y, %H:%M:%S")), end=" --- ")
            model = LogisticRegression(penalty = penalty,
                                         fit_intercept = fit_intercept,
                                         class_weight = class_weight,
                                         solver = solver)
            model.fit(x_train,y_train)
            print("End: {}".format(datetime.now().strftime("%m/%d/%Y, %H:%M:%S")))
        
            # Testing
            print("\t\tValidation: {}".format(fold_index), end = " -- ")
            print("Start: {}".format(datetime.now().strftime("%m/%d/%Y, %H:%M:%S")), end="---")
            y_pred = model.predict(x_test)
            print("End: {}".format(datetime.now().strftime("%m/%d/%Y, %H:%M:%S")))
        
            # Evaluation
            cm = confusion_matrix(y_test, y_pred)
            current_utils = utils(cm, y_test, y_pred)
            output_dict["fold_{}".format(fold_index)] = current_utils.get_value()
            
            fold_index += 1
        print("\n")
        return output_dict 
    
    def train(self, path):
        count = 0
        for x in self.param_grid["penalty"]:
            for i in self.param_grid["fit_intercept"]:
                for j in self.param_grid["class_weight"]:
                    for k in self.param_grid["solver"]:
                        print("Traning Case: {}".format(count))
                        self.history["train_{}".format(count)] = {
                            "param" : {
                            "penalty" : x,
                            "fit_intercept" : i,
                            "class_weight" : j,
                            "solver" : k,
                            },
                            "train_fold" : self.training(x, i, j, k, count)
                        }
                        count += 1
        with open(path.format(self.fold_count), 'wb') as f:
            pickle.dump(self.history, f)
        
        return self.history

# Training

In [11]:
training = Training(10, kmean_train_paths, kmean_test_paths)
approach_dir = os.getcwd() + "\\Experiment\\Approach"
svm_dir = approach_dir + "\\LogisticRegression"
save_result_path = svm_dir + '\\lr_km64_akaze_result_fold{0}_0.9.pkl'
training.train(path = save_result_path)

Traning Case: 0
	Fold: 0
	TRAIN: D:\Github\Stomach-Status-Classification\Data set\kmean_dataset\10cv_64\akaze_train_0_3_0.001_4_4histo_64_cv_index0.csv 
	TEST: D:\Github\Stomach-Status-Classification\Data set\kmean_dataset\10cv_64\akaze_test_0_3_0.001_4_4histo_64_cv_index0.csv
		Training : 0 -- Start: 11/14/2022, 16:24:21 --- End: 11/14/2022, 16:24:21
		Validation: 0 -- Start: 11/14/2022, 16:24:21---End: 11/14/2022, 16:24:21
	Fold: 1
	TRAIN: D:\Github\Stomach-Status-Classification\Data set\kmean_dataset\10cv_64\akaze_train_0_3_0.001_4_4histo_64_cv_index1.csv 
	TEST: D:\Github\Stomach-Status-Classification\Data set\kmean_dataset\10cv_64\akaze_test_0_3_0.001_4_4histo_64_cv_index1.csv
		Training : 1 -- Start: 11/14/2022, 16:24:21 --- End: 11/14/2022, 16:24:21
		Validation: 1 -- Start: 11/14/2022, 16:24:21---End: 11/14/2022, 16:24:21
	Fold: 2
	TRAIN: D:\Github\Stomach-Status-Classification\Data set\kmean_dataset\10cv_64\akaze_train_0_3_0.001_4_4histo_64_cv_index2.csv 
	TEST: D:\Github\Stom

C:\Users\kohi2\AppData\Local\Temp\ipykernel_7068\425510141.py:25: RuntimeWarning: divide by zero encountered in double_scalars
  self.positive_likelihood_ratio = self.sensitivity / self.false_positive_rate # Positive Likelihood Ratio
C:\Users\kohi2\AppData\Local\Temp\ipykernel_7068\425510141.py:25: RuntimeWarning: divide by zero encountered in double_scalars
  self.positive_likelihood_ratio = self.sensitivity / self.false_positive_rate # Positive Likelihood Ratio


End: 11/14/2022, 16:24:21
		Validation: 8 -- Start: 11/14/2022, 16:24:21---End: 11/14/2022, 16:24:21
	Fold: 9
	TRAIN: D:\Github\Stomach-Status-Classification\Data set\kmean_dataset\10cv_64\akaze_train_0_3_0.001_4_4histo_64_cv_index9.csv 
	TEST: D:\Github\Stomach-Status-Classification\Data set\kmean_dataset\10cv_64\akaze_test_0_3_0.001_4_4histo_64_cv_index9.csv
		Training : 9 -- Start: 11/14/2022, 16:24:21 --- End: 11/14/2022, 16:24:21
		Validation: 9 -- Start: 11/14/2022, 16:24:21---End: 11/14/2022, 16:24:21


Traning Case: 1
	Fold: 0
	TRAIN: D:\Github\Stomach-Status-Classification\Data set\kmean_dataset\10cv_64\akaze_train_0_3_0.001_4_4histo_64_cv_index0.csv 
	TEST: D:\Github\Stomach-Status-Classification\Data set\kmean_dataset\10cv_64\akaze_test_0_3_0.001_4_4histo_64_cv_index0.csv
		Training : 0 -- Start: 11/14/2022, 16:24:21 --- End: 11/14/2022, 16:24:21
		Validation: 0 -- Start: 11/14/2022, 16:24:21---End: 11/14/2022, 16:24:21
	Fold: 1
	TRAIN: D:\Github\Stomach-Status-Classificatio

C:\Users\kohi2\AppData\Local\Temp\ipykernel_7068\425510141.py:25: RuntimeWarning: divide by zero encountered in double_scalars
  self.positive_likelihood_ratio = self.sensitivity / self.false_positive_rate # Positive Likelihood Ratio
C:\Users\kohi2\AppData\Local\Temp\ipykernel_7068\425510141.py:25: RuntimeWarning: divide by zero encountered in double_scalars
  self.positive_likelihood_ratio = self.sensitivity / self.false_positive_rate # Positive Likelihood Ratio
C:\Users\kohi2\AppData\Local\Temp\ipykernel_7068\425510141.py:25: RuntimeWarning: divide by zero encountered in double_scalars
  self.positive_likelihood_ratio = self.sensitivity / self.false_positive_rate # Positive Likelihood Ratio


End: 11/14/2022, 16:24:21
		Validation: 7 -- Start: 11/14/2022, 16:24:21---End: 11/14/2022, 16:24:21
	Fold: 8
	TRAIN: D:\Github\Stomach-Status-Classification\Data set\kmean_dataset\10cv_64\akaze_train_0_3_0.001_4_4histo_64_cv_index8.csv 
	TEST: D:\Github\Stomach-Status-Classification\Data set\kmean_dataset\10cv_64\akaze_test_0_3_0.001_4_4histo_64_cv_index8.csv
		Training : 8 -- Start: 11/14/2022, 16:24:21 --- End: 11/14/2022, 16:24:21
		Validation: 8 -- Start: 11/14/2022, 16:24:21---End: 11/14/2022, 16:24:21
	Fold: 9
	TRAIN: D:\Github\Stomach-Status-Classification\Data set\kmean_dataset\10cv_64\akaze_train_0_3_0.001_4_4histo_64_cv_index9.csv 
	TEST: D:\Github\Stomach-Status-Classification\Data set\kmean_dataset\10cv_64\akaze_test_0_3_0.001_4_4histo_64_cv_index9.csv
		Training : 9 -- Start: 11/14/2022, 16:24:21 --- End: 11/14/2022, 16:24:21
		Validation: 9 -- Start: 11/14/2022, 16:24:21---End: 11/14/2022, 16:24:21


Traning Case: 2
	Fold: 0
	TRAIN: D:\Github\Stomach-Status-Classificatio

...

d:\github\stomach-status-classification\envir\lib\site-packages\sklearn\linear_model\_sag.py:350: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn(
C:\Users\kohi2\AppData\Local\Temp\ipykernel_7068\425510141.py:25: RuntimeWarning: divide by zero encountered in double_scalars
  self.positive_likelihood_ratio = self.sensitivity / self.false_positive_rate # Positive Likelihood Ratio
d:\github\stomach-status-classification\envir\lib\site-packages\sklearn\linear_model\_sag.py:350: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn(
d:\github\stomach-status-classification\envir\lib\site-packages\sklearn\linear_model\_sag.py:350: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn(
d:\github\stomach-status-classification\envir\lib\site-packages\sklearn\linear_model\_sag.py:350: ConvergenceWarning: The max_iter was reached which means the coef_ did 

	Fold: 9
	TRAIN: D:\Github\Stomach-Status-Classification\Data set\kmean_dataset\10cv_64\akaze_train_0_3_0.001_4_4histo_64_cv_index9.csv 
	TEST: D:\Github\Stomach-Status-Classification\Data set\kmean_dataset\10cv_64\akaze_test_0_3_0.001_4_4histo_64_cv_index9.csv
		Training : 9 -- Start: 11/14/2022, 16:24:25 --- End: 11/14/2022, 16:24:25
		Validation: 9 -- Start: 11/14/2022, 16:24:25---End: 11/14/2022, 16:24:25


Traning Case: 19
	Fold: 0
	TRAIN: D:\Github\Stomach-Status-Classification\Data set\kmean_dataset\10cv_64\akaze_train_0_3_0.001_4_4histo_64_cv_index0.csv 
	TEST: D:\Github\Stomach-Status-Classification\Data set\kmean_dataset\10cv_64\akaze_test_0_3_0.001_4_4histo_64_cv_index0.csv
		Training : 0 -- Start: 11/14/2022, 16:24:25 --- End: 11/14/2022, 16:24:25
		Validation: 0 -- Start: 11/14/2022, 16:24:25---End: 11/14/2022, 16:24:25
	Fold: 1
	TRAIN: D:\Github\Stomach-Status-Classification\Data set\kmean_dataset\10cv_64\akaze_train_0_3_0.001_4_4histo_64_cv_index1.csv 
	TEST: D:\Github\S

d:\github\stomach-status-classification\envir\lib\site-packages\sklearn\linear_model\_sag.py:350: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn(
C:\Users\kohi2\AppData\Local\Temp\ipykernel_7068\425510141.py:25: RuntimeWarning: divide by zero encountered in double_scalars
  self.positive_likelihood_ratio = self.sensitivity / self.false_positive_rate # Positive Likelihood Ratio
d:\github\stomach-status-classification\envir\lib\site-packages\sklearn\linear_model\_sag.py:350: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn(
d:\github\stomach-status-classification\envir\lib\site-packages\sklearn\linear_model\_sag.py:350: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn(
d:\github\stomach-status-classification\envir\lib\site-packages\sklearn\linear_model\_sag.py:350: ConvergenceWarning: The max_iter was reached which means the coef_ did 

	Fold: 6
	TRAIN: D:\Github\Stomach-Status-Classification\Data set\kmean_dataset\10cv_64\akaze_train_0_3_0.001_4_4histo_64_cv_index6.csv 
	TEST: D:\Github\Stomach-Status-Classification\Data set\kmean_dataset\10cv_64\akaze_test_0_3_0.001_4_4histo_64_cv_index6.csv
		Training : 6 -- Start: 11/14/2022, 16:24:26 --- End: 11/14/2022, 16:24:26
		Validation: 6 -- Start: 11/14/2022, 16:24:26---End: 11/14/2022, 16:24:26
	Fold: 7
	TRAIN: D:\Github\Stomach-Status-Classification\Data set\kmean_dataset\10cv_64\akaze_train_0_3_0.001_4_4histo_64_cv_index7.csv 
	TEST: D:\Github\Stomach-Status-Classification\Data set\kmean_dataset\10cv_64\akaze_test_0_3_0.001_4_4histo_64_cv_index7.csv
		Training : 7 -- Start: 11/14/2022, 16:24:26 --- End: 11/14/2022, 16:24:26
		Validation: 7 -- Start: 11/14/2022, 16:24:26---End: 11/14/2022, 16:24:26
	Fold: 8
	TRAIN: D:\Github\Stomach-Status-Classification\Data set\kmean_dataset\10cv_64\akaze_train_0_3_0.001_4_4histo_64_cv_index8.csv 
	TEST: D:\Github\Stomach-Status-Class

{'train_0': {'param': {'penalty': 'l2',
   'fit_intercept': True,
   'class_weight': 'balanced',
   'solver': 'newton-cg'},
  'train_fold': {'fold_0': {'Confusion Matrix': {'TP': 2,
     'TN': 12,
     'FN': 0,
     'FP': 0,
     'precision': 1.0,
     'recall': 1.0,
     'f1_score': 1.0,
     'sensitivity': 1.0,
     'specificity': 1.0,
     'negative_predictive_value': 1.0,
     'false_negative_rate': 0.0,
     'false_positive_rate': 0.0,
     'false_discovery_rate': 0.0,
     'false_omission_rate': 0.0,
     'Positive_likelihood_ratio': inf,
     'Negative_likelihood_ratio': 0.0,
     'prevalence_threshold': 0.0,
     'threat_score': 0.16666666666666666,
     'Prevalence': 0.14285714285714285,
     'Matthews_correlation_coefficient': 0.041666666666666664,
     'Fowlkes_Mallows_index': 1.4142135623730951,
     'informedness': 1.0,
     'markedness': 1.0,
     'Diagnostic_odds_ratio': inf,
     'accuracy': 1.0,
     'balanced_accuracy': 1.0},
    'ROC_AUC_SCORE': {'Macro': 0.7, 'Micro

# Result Analyzing

In [12]:
file = open(save_result_path.format(10), 'rb')
result = pickle.load(file)
result

{'train_0': {'param': {'penalty': 'l2',
   'fit_intercept': True,
   'class_weight': 'balanced',
   'solver': 'newton-cg'},
  'train_fold': {'fold_0': {'Confusion Matrix': {'TP': 2,
     'TN': 12,
     'FN': 0,
     'FP': 0,
     'precision': 1.0,
     'recall': 1.0,
     'f1_score': 1.0,
     'sensitivity': 1.0,
     'specificity': 1.0,
     'negative_predictive_value': 1.0,
     'false_negative_rate': 0.0,
     'false_positive_rate': 0.0,
     'false_discovery_rate': 0.0,
     'false_omission_rate': 0.0,
     'Positive_likelihood_ratio': inf,
     'Negative_likelihood_ratio': 0.0,
     'prevalence_threshold': 0.0,
     'threat_score': 0.16666666666666666,
     'Prevalence': 0.14285714285714285,
     'Matthews_correlation_coefficient': 0.041666666666666664,
     'Fowlkes_Mallows_index': 1.4142135623730951,
     'informedness': 1.0,
     'markedness': 1.0,
     'Diagnostic_odds_ratio': inf,
     'accuracy': 1.0,
     'balanced_accuracy': 1.0},
    'ROC_AUC_SCORE': {'Macro': 0.7, 'Micro

In [13]:
print("Number of Training: {}".format(len(list(result.keys()))))

Number of Training: 20


In [14]:
training_case_lst = list(result.keys())
print(training_case_lst[0])

train_0


In [15]:
train_sample = result[training_case_lst[0]]
print(train_sample['param'])
print("Number of Fold: {}".format(len(train_sample['train_fold'])))

{'penalty': 'l2', 'fit_intercept': True, 'class_weight': 'balanced', 'solver': 'newton-cg'}
Number of Fold: 10


In [16]:
fold_sample = train_sample['train_fold']['fold_0']
print(fold_sample['Classification Report'])

{'0': {'precision': 1.0, 'recall': 0.4, 'f1-score': 0.5714285714285715, 'support': 5}, '1': {'precision': 0.8, 'recall': 1.0, 'f1-score': 0.888888888888889, 'support': 12}, 'accuracy': 0.8235294117647058, 'macro avg': {'precision': 0.9, 'recall': 0.7, 'f1-score': 0.7301587301587302, 'support': 17}, 'weighted avg': {'precision': 0.8588235294117648, 'recall': 0.8235294117647058, 'f1-score': 0.7955182072829132, 'support': 17}}


In [17]:
result_analyze_dict = {
    'Train' : [],
    'MACC' : [],
    'MAP' : [],
    'MASens' : [],
    'MASpec' : [],
    'MAF1' : [],
    'AUC' : []
}

for result_key in result:
    result_analyze_dict['Train'].append(result_key)
    train_base = result[result_key]
    fold_base = train_base['train_fold']
    acc_lst, pre_lst, re_lst, spec_lst, f1_lst, auc_lst = [], [], [], [], [], []
    for fold_key in fold_base:
        current_fold = fold_base[fold_key]
        current_fold_cls = current_fold['Classification Report']
        acc_lst.append(current_fold_cls['accuracy'])
        pre_lst.append(current_fold_cls['macro avg']['precision'])
        re_lst.append(current_fold_cls['macro avg']['recall'])
        spec_lst.append(current_fold['Confusion Matrix']['specificity'])
        auc_lst.append(current_fold['ROC_AUC_SCORE']['Macro'])
        f1_lst.append(current_fold_cls['macro avg']['f1-score'])
    result_analyze_dict['MACC'].append(sum(acc_lst)/len(acc_lst))
    result_analyze_dict['MAP'].append(sum(pre_lst)/len(pre_lst))
    result_analyze_dict['MASens'].append(sum(re_lst)/len(re_lst))
    result_analyze_dict['MASpec'].append(sum(spec_lst)/len(spec_lst))
    result_analyze_dict['MAF1'].append(sum(f1_lst)/len(f1_lst))
    result_analyze_dict['AUC'].append(sum(auc_lst)/len(auc_lst))

result_df = pd.DataFrame(result_analyze_dict)
result_df.head()

,Train,MACC,MAP,MASens,MASpec,MAF1,AUC
0,train_0,0.757353,0.735668,0.648333,0.916667,0.657005,0.648333
1,train_1,0.757353,0.735668,0.648333,0.916667,0.657005,0.648333
2,train_2,0.757353,0.735668,0.648333,0.916667,0.657005,0.648333
3,train_3,0.751471,0.722619,0.644167,0.908333,0.650684,0.644167
4,train_4,0.769118,0.739905,0.697500,0.875000,0.691798,0.697500


In [18]:
result_max = result_df.max()
result_max

Train      train_9
MACC        0.7875
MAP       0.766445
MASens        0.78
MASpec    0.991667
MAF1      0.761309
AUC           0.78
dtype: object

In [19]:
id_result_max = result_df.iloc[:, 1:].idxmax()
id_result_max

MACC      10
MAP       10
MASens    10
MASpec     8
MAF1      10
AUC       10
dtype: int64

In [20]:
id_max = 'train_{}'.format(id_result_max.mode()[0])
print(id_max)

train_10


In [21]:
result_df.loc[result_df['Train'] == id_max]

,Train,MACC,MAP,MASens,MASpec,MAF1,AUC
10,train_10,0.7875,0.766445,0.78,0.8,0.761309,0.78


In [22]:
train_max = result[id_max]
train_max['param']

{'penalty': 'l2',
 'fit_intercept': False,
 'class_weight': 'balanced',
 'solver': 'newton-cg'}

In [23]:
train_max_fold = train_max['train_fold']
train_max_analyze = {
    'Fold' : [],
    'ACC' : [],
    'AP' : [],
    'ASens' : [],
    'ASpec' : [],
    'AF1' : [],
    'AUC' : []
}

for fold_key in train_max_fold:
    train_max_analyze['Fold'].append(fold_key)
    current_fold = train_max_fold[fold_key]
    current_fold_cls = current_fold['Classification Report']
    train_max_analyze['ACC'].append(current_fold_cls['accuracy'])
    train_max_analyze['AP'].append(current_fold_cls['macro avg']['precision'])
    train_max_analyze['ASens'].append(current_fold_cls['macro avg']['recall'])
    train_max_analyze['ASpec'].append(current_fold['Confusion Matrix']['specificity'])
    train_max_analyze['AF1'].append(current_fold_cls['macro avg']['f1-score'])
    train_max_analyze['AUC'].append(current_fold['ROC_AUC_SCORE']['Macro'])

train_max_analyze_df = pd.DataFrame(train_max_analyze)
train_max_analyze_df

,Fold,ACC,AP,ASens,ASpec,AF1,AUC
0,fold_0,0.705882,0.694444,0.733333,0.666667,0.688645,0.733333
1,fold_1,0.705882,0.659091,0.675000,0.750000,0.664032,0.675000
2,fold_2,0.823529,0.787879,0.816667,0.833333,0.798419,0.816667
3,fold_3,0.882353,0.858333,0.858333,0.916667,0.858333,0.858333
4,fold_4,0.823529,0.787879,0.816667,0.833333,0.798419,0.816667
5,fold_5,0.882353,0.857143,0.916667,0.833333,0.871212,0.916667
6,fold_6,0.705882,0.694444,0.733333,0.666667,0.688645,0.733333
7,fold_7,0.823529,0.787879,0.816667,0.833333,0.798419,0.816667
8,fold_8,0.647059,0.614286,0.633333,0.666667,0.613636,0.633333
9,fold_9,0.875000,0.923077,0.800000,1.000000,0.833333,0.800000


# N_times_K_folds

In [24]:
# Setup

train_max_param = train_max['param']
n_times = 100

nt_kf_output_dict = {
    'Train' : [], # <index>_<fold>
    'ACC' : [],
    'AP' : [],
    'ASens' : [],
    'ASpec' : [],
    'AF1' : [],
    'AUC' : []
}

for n in range(n_times):
    print("Training {}".format(n))
    fold_index = 0
    for train_index, test_index in zip(kmean_train_paths, kmean_test_paths):
        train_df = pd.read_csv(train_index)
        test_df = pd.read_csv(test_index)      
    
        print("\tFold: {}".format(fold_index))
        print("\tTRAIN:", train_index, "\n\tTEST:", test_index)

        # folding data
        x_train, y_train = train_df.iloc[:, 2:-1], train_df.iloc[:, -1]
        x_test, y_test = test_df.iloc[:, 2:-1], test_df.iloc[:, -1]
        
        # Training
        print("\t\tTraining : {}".format(fold_index), end = " -- ")
        print("Start: {}".format(datetime.now().strftime("%m/%d/%Y, %H:%M:%S")), end=" --- ")
        model = LogisticRegression(penalty = train_max_param['penalty'],
                                 fit_intercept = train_max_param['fit_intercept'],
                                 class_weight = train_max_param['class_weight'],
                                 solver = train_max_param['solver'])
        model.fit(x_train,y_train)
        print("End: {}".format(datetime.now().strftime("%m/%d/%Y, %H:%M:%S")))

        # Testing
        print("\t\tValidation: {}".format(fold_index), end = " -- ")
        print("Start: {}".format(datetime.now().strftime("%m/%d/%Y, %H:%M:%S")), end="---")
        y_pred = model.predict(x_test)
        print("End: {}".format(datetime.now().strftime("%m/%d/%Y, %H:%M:%S")))
        
        # Evaluation
        cm = confusion_matrix(y_test, y_pred)
        base_result = utils(cm, y_test, y_pred).get_value()
        
        # Result Storing
        nt_kf_output_dict['Train'].append('{0}_{1}'.format(n, fold_index))
        cls = base_result['Classification Report']
        nt_kf_output_dict['ACC'].append(cls['accuracy'])
        nt_kf_output_dict['AP'].append(cls['macro avg']['precision'])
        nt_kf_output_dict['ASens'].append(cls['macro avg']['recall'])
        nt_kf_output_dict['ASpec'].append(base_result['Confusion Matrix']['specificity'])
        nt_kf_output_dict['AF1'].append(cls['macro avg']['f1-score'])
        nt_kf_output_dict['AUC'].append(base_result['ROC_AUC_SCORE']['Macro'])
        
        # Fold index increment
        fold_index += 1

Training 0
	Fold: 0
	TRAIN: D:\Github\Stomach-Status-Classification\Data set\kmean_dataset\10cv_64\akaze_train_0_3_0.001_4_4histo_64_cv_index0.csv 
	TEST: D:\Github\Stomach-Status-Classification\Data set\kmean_dataset\10cv_64\akaze_test_0_3_0.001_4_4histo_64_cv_index0.csv
		Training : 0 -- Start: 11/14/2022, 16:24:27 --- End: 11/14/2022, 16:24:27
		Validation: 0 -- Start: 11/14/2022, 16:24:27---End: 11/14/2022, 16:24:27
	Fold: 1
	TRAIN: D:\Github\Stomach-Status-Classification\Data set\kmean_dataset\10cv_64\akaze_train_0_3_0.001_4_4histo_64_cv_index1.csv 
	TEST: D:\Github\Stomach-Status-Classification\Data set\kmean_dataset\10cv_64\akaze_test_0_3_0.001_4_4histo_64_cv_index1.csv
		Training : 1 -- Start: 11/14/2022, 16:24:27 --- End: 11/14/2022, 16:24:27
		Validation: 1 -- Start: 11/14/2022, 16:24:27---End: 11/14/2022, 16:24:27
	Fold: 2
	TRAIN: D:\Github\Stomach-Status-Classification\Data set\kmean_dataset\10cv_64\akaze_train_0_3_0.001_4_4histo_64_cv_index2.csv 
	TEST: D:\Github\Stomach-S

C:\Users\kohi2\AppData\Local\Temp\ipykernel_7068\425510141.py:25: RuntimeWarning: divide by zero encountered in double_scalars
  self.positive_likelihood_ratio = self.sensitivity / self.false_positive_rate # Positive Likelihood Ratio


	Fold: 7
	TRAIN: D:\Github\Stomach-Status-Classification\Data set\kmean_dataset\10cv_64\akaze_train_0_3_0.001_4_4histo_64_cv_index7.csv 
	TEST: D:\Github\Stomach-Status-Classification\Data set\kmean_dataset\10cv_64\akaze_test_0_3_0.001_4_4histo_64_cv_index7.csv
		Training : 7 -- Start: 11/14/2022, 16:24:27 --- End: 11/14/2022, 16:24:27
		Validation: 7 -- Start: 11/14/2022, 16:24:27---End: 11/14/2022, 16:24:27
	Fold: 8
	TRAIN: D:\Github\Stomach-Status-Classification\Data set\kmean_dataset\10cv_64\akaze_train_0_3_0.001_4_4histo_64_cv_index8.csv 
	TEST: D:\Github\Stomach-Status-Classification\Data set\kmean_dataset\10cv_64\akaze_test_0_3_0.001_4_4histo_64_cv_index8.csv
		Training : 8 -- Start: 11/14/2022, 16:24:27 --- End: 11/14/2022, 16:24:27
		Validation: 8 -- Start: 11/14/2022, 16:24:27---End: 11/14/2022, 16:24:27
	Fold: 9
	TRAIN: D:\Github\Stomach-Status-Classification\Data set\kmean_dataset\10cv_64\akaze_train_0_3_0.001_4_4histo_64_cv_index9.csv 
	TEST: D:\Github\Stomach-Status-Class

C:\Users\kohi2\AppData\Local\Temp\ipykernel_7068\425510141.py:25: RuntimeWarning: divide by zero encountered in double_scalars
  self.positive_likelihood_ratio = self.sensitivity / self.false_positive_rate # Positive Likelihood Ratio


	Fold: 7
	TRAIN: D:\Github\Stomach-Status-Classification\Data set\kmean_dataset\10cv_64\akaze_train_0_3_0.001_4_4histo_64_cv_index7.csv 
	TEST: D:\Github\Stomach-Status-Classification\Data set\kmean_dataset\10cv_64\akaze_test_0_3_0.001_4_4histo_64_cv_index7.csv
		Training : 7 -- Start: 11/14/2022, 16:24:28 --- End: 11/14/2022, 16:24:28
		Validation: 7 -- Start: 11/14/2022, 16:24:28---End: 11/14/2022, 16:24:28
	Fold: 8
	TRAIN: D:\Github\Stomach-Status-Classification\Data set\kmean_dataset\10cv_64\akaze_train_0_3_0.001_4_4histo_64_cv_index8.csv 
	TEST: D:\Github\Stomach-Status-Classification\Data set\kmean_dataset\10cv_64\akaze_test_0_3_0.001_4_4histo_64_cv_index8.csv
		Training : 8 -- Start: 11/14/2022, 16:24:28 --- End: 11/14/2022, 16:24:28
		Validation: 8 -- Start: 11/14/2022, 16:24:28---End: 11/14/2022, 16:24:28
	Fold: 9
	TRAIN: D:\Github\Stomach-Status-Classification\Data set\kmean_dataset\10cv_64\akaze_train_0_3_0.001_4_4histo_64_cv_index9.csv 
	TEST: D:\Github\Stomach-Status-Class

...

C:\Users\kohi2\AppData\Local\Temp\ipykernel_7068\425510141.py:25: RuntimeWarning: divide by zero encountered in double_scalars
  self.positive_likelihood_ratio = self.sensitivity / self.false_positive_rate # Positive Likelihood Ratio


End: 11/14/2022, 16:24:51
		Validation: 6 -- Start: 11/14/2022, 16:24:51---End: 11/14/2022, 16:24:51
	Fold: 7
	TRAIN: D:\Github\Stomach-Status-Classification\Data set\kmean_dataset\10cv_64\akaze_train_0_3_0.001_4_4histo_64_cv_index7.csv 
	TEST: D:\Github\Stomach-Status-Classification\Data set\kmean_dataset\10cv_64\akaze_test_0_3_0.001_4_4histo_64_cv_index7.csv
		Training : 7 -- Start: 11/14/2022, 16:24:51 --- End: 11/14/2022, 16:24:51
		Validation: 7 -- Start: 11/14/2022, 16:24:51---End: 11/14/2022, 16:24:51
	Fold: 8
	TRAIN: D:\Github\Stomach-Status-Classification\Data set\kmean_dataset\10cv_64\akaze_train_0_3_0.001_4_4histo_64_cv_index8.csv 
	TEST: D:\Github\Stomach-Status-Classification\Data set\kmean_dataset\10cv_64\akaze_test_0_3_0.001_4_4histo_64_cv_index8.csv
		Training : 8 -- Start: 11/14/2022, 16:24:51 --- End: 11/14/2022, 16:24:51
		Validation: 8 -- Start: 11/14/2022, 16:24:51---End: 11/14/2022, 16:24:51
	Fold: 9
	TRAIN: D:\Github\Stomach-Status-Classification\Data set\kmean_d

C:\Users\kohi2\AppData\Local\Temp\ipykernel_7068\425510141.py:25: RuntimeWarning: divide by zero encountered in double_scalars
  self.positive_likelihood_ratio = self.sensitivity / self.false_positive_rate # Positive Likelihood Ratio


End: 11/14/2022, 16:24:51
		Validation: 5 -- Start: 11/14/2022, 16:24:51---End: 11/14/2022, 16:24:51
	Fold: 6
	TRAIN: D:\Github\Stomach-Status-Classification\Data set\kmean_dataset\10cv_64\akaze_train_0_3_0.001_4_4histo_64_cv_index6.csv 
	TEST: D:\Github\Stomach-Status-Classification\Data set\kmean_dataset\10cv_64\akaze_test_0_3_0.001_4_4histo_64_cv_index6.csv
		Training : 6 -- Start: 11/14/2022, 16:24:51 --- End: 11/14/2022, 16:24:51
		Validation: 6 -- Start: 11/14/2022, 16:24:51---End: 11/14/2022, 16:24:51
	Fold: 7
	TRAIN: D:\Github\Stomach-Status-Classification\Data set\kmean_dataset\10cv_64\akaze_train_0_3_0.001_4_4histo_64_cv_index7.csv 
	TEST: D:\Github\Stomach-Status-Classification\Data set\kmean_dataset\10cv_64\akaze_test_0_3_0.001_4_4histo_64_cv_index7.csv
		Training : 7 -- Start: 11/14/2022, 16:24:51 --- End: 11/14/2022, 16:24:51
		Validation: 7 -- Start: 11/14/2022, 16:24:51---End: 11/14/2022, 16:24:51
	Fold: 8
	TRAIN: D:\Github\Stomach-Status-Classification\Data set\kmean_d

C:\Users\kohi2\AppData\Local\Temp\ipykernel_7068\425510141.py:25: RuntimeWarning: divide by zero encountered in double_scalars
  self.positive_likelihood_ratio = self.sensitivity / self.false_positive_rate # Positive Likelihood Ratio


In [25]:
nt_kf_output_df = pd.DataFrame(nt_kf_output_dict)
nt_kf_output_df.head(10)

,Train,ACC,AP,ASens,ASpec,AF1,AUC
0,0_0,0.705882,0.694444,0.733333,0.666667,0.688645,0.733333
1,0_1,0.705882,0.659091,0.675000,0.750000,0.664032,0.675000
2,0_2,0.823529,0.787879,0.816667,0.833333,0.798419,0.816667
3,0_3,0.882353,0.858333,0.858333,0.916667,0.858333,0.858333
4,0_4,0.823529,0.787879,0.816667,0.833333,0.798419,0.816667
5,0_5,0.882353,0.857143,0.916667,0.833333,0.871212,0.916667
6,0_6,0.705882,0.694444,0.733333,0.666667,0.688645,0.733333
7,0_7,0.823529,0.787879,0.816667,0.833333,0.798419,0.816667
8,0_8,0.647059,0.614286,0.633333,0.666667,0.613636,0.633333
9,0_9,0.875000,0.923077,0.800000,1.000000,0.833333,0.800000


In [26]:
nt_kf_output_df.mean()

Train         inf
ACC      0.787500
AP       0.766445
ASens    0.780000
ASpec    0.800000
AF1      0.761309
AUC      0.780000
dtype: float64

In [27]:
nt_kf_output_df.std(numeric_only=True)

ACC      0.083260
AP       0.093656
ASens    0.081367
ASpec    0.106772
AF1      0.085273
AUC      0.081367
dtype: float64

In [28]:
acc_values = nt_kf_output_df['ACC'].values.tolist()
ap_values = nt_kf_output_df['AP'].values.tolist()
asens_values = nt_kf_output_df['ASens'].values.tolist()
aspec_values = nt_kf_output_df['ASpec'].values.tolist()
af1_values = nt_kf_output_df['AF1'].values.tolist()
auc_values = nt_kf_output_df['AUC'].values.tolist()
print(len(acc_values), len(ap_values), len(asens_values), len(aspec_values), len(af1_values), len(auc_values))

1000 1000 1000 1000 1000 1000


In [29]:
import scipy.stats as st

def CI(data, confidence_level, dis_type = 't'):
    if dis_type == 't':
        return st.t.interval(confidence=confidence_level, df=len(data)-1, 
                             loc=np.mean(data), scale=st.sem(data))
    elif dis_type == 'g':
        return st.norm.interval(confidence=confidence_level, loc=np.mean(data), scale=st.sem(data))

def CI_calculator(confidence_level, dis_type = 't'):
    for x in nt_kf_output_df.columns[1:]:
        base = nt_kf_output_df[x].values.tolist()
        print("CI of {}".format(x), CI(base, confidence_level, dis_type))

In [30]:
cl = 0.95

CI_calculator(confidence_level = cl, dis_type = 't')

CI of ACC (0.7823333321818814, 0.7926666678181186)
CI of AP (0.7606337017131298, 0.7722572961778681)
CI of ASens (0.7749508266290877, 0.7850491733709124)
CI of ASpec (0.7933742886210441, 0.806625711378956)
CI of AF1 (0.756017734109252, 0.7666008791615349)
CI of AUC (0.7749508266290877, 0.7850491733709124)


In [31]:
cl = 0.95

CI_calculator(confidence_level = cl, dis_type = 'g')

CI of ACC (0.7823395918630056, 0.7926604081369943)
CI of AP (0.7606407430013863, 0.7722502548896116)
CI of ASens (0.7749569439597067, 0.7850430560402933)
CI of ASpec (0.7933823160077299, 0.8066176839922702)
CI of AF1 (0.7560241451187895, 0.7665944681519974)
CI of AUC (0.7749569439597067, 0.7850430560402933)


In [32]:
cl = 0.99

CI_calculator(confidence_level = cl, dis_type = 'g')

CI of ACC (0.7807180741062639, 0.794281925893736)
CI of AP (0.7588167564896503, 0.7740742414013476)
CI of ASens (0.7733723009042561, 0.7866276990957439)
CI of ASpec (0.7913028889901178, 0.8086971110098823)
CI of AF1 (0.7543634270091144, 0.7682551862616724)
CI of AUC (0.7733723009042561, 0.7866276990957439)
